In [1]:
import keras
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [2]:
df = pd.read_csv("adr_data.csv")
df.head()

,hotel,is_canceled,lead_time,arrival_date_week_number,stays_in_weekend_nights,stays_in_week_nights,adults,children,is_repeated_guest,previous_cancellations,...,agent_468.0,agent_469.0,agent_476.0,agent_481.0,agent_510.0,agent_0,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party
0,1,0,257,27,0,2,1,0.0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,0,257,27,0,2,2,0.0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1,0,257,27,0,2,2,0.0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,0,257,27,0,2,2,0.0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1,0,257,27,0,2,2,0.0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [3]:
X = df.drop(["adr"], axis=1).values
y = df["adr"].values

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1126)

In [5]:
earlystop = EarlyStopping(monitor='loss', patience=5)
reducelr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

In [23]:
# create model
model = Sequential()
model.add(Dense(256, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(512, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1024, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1024, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(512, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, kernel_initializer='normal',activation='linear'))
# Compile model
model.compile(loss='mean_squared_error', optimizer='nadam')

In [24]:
model.fit(X_train, y_train, epochs=500, batch_size=32, validation_split=0.2, callbacks=[earlystop, reducelr])

Epoch 1/500
1144/1144 [==============================] - 3s 2ms/step - loss: 1797.3533 - val_loss: 1338.3085
Epoch 2/500
1144/1144 [==============================] - 3s 2ms/step - loss: 1144.0397 - val_loss: 1122.3627
Epoch 3/500
1144/1144 [==============================] - 3s 2ms/step - loss: 997.9726 - val_loss: 1214.4640
Epoch 4/500
1144/1144 [==============================] - 3s 2ms/step - loss: 886.2108 - val_loss: 749.3038
Epoch 5/500
1144/1144 [==============================] - 3s 2ms/step - loss: 803.6004 - val_loss: 1346.9327
Epoch 6/500
1144/1144 [==============================] - 3s 2ms/step - loss: 749.7192 - val_loss: 812.5149
Epoch 7/500
1144/1144 [==============================] - 3s 2ms/step - loss: 702.8766 - val_loss: 685.4554
Epoch 8/500
1144/1144 [==============================] - 3s 2ms/step - loss: 668.7688 - val_loss: 612.8658
Epoch 9/500
1144/1144 [==============================] - 3s 2ms/step - loss: 638.2866 - val_loss: 767.4251
Epoch 10/500
1144/1144 [=======

Epoch 77/500
1144/1144 [==============================] - 3s 2ms/step - loss: 373.2245 - val_loss: 432.9506
Epoch 78/500
1144/1144 [==============================] - 3s 2ms/step - loss: 371.9934 - val_loss: 464.1468
Epoch 79/500
1144/1144 [==============================] - 3s 2ms/step - loss: 369.7293 - val_loss: 478.3865
Epoch 80/500
1144/1144 [==============================] - 3s 2ms/step - loss: 371.9784 - val_loss: 453.0992
Epoch 81/500
1144/1144 [==============================] - 3s 2ms/step - loss: 368.7101 - val_loss: 461.0887
Epoch 82/500
1144/1144 [==============================] - 3s 2ms/step - loss: 365.4173 - val_loss: 481.0366
Epoch 83/500
1144/1144 [==============================] - 3s 2ms/step - loss: 364.2008 - val_loss: 428.9937
Epoch 84/500
1144/1144 [==============================] - 3s 2ms/step - loss: 368.1138 - val_loss: 442.5085
Epoch 85/500
1144/1144 [==============================] - 3s 2ms/step - loss: 364.7442 - val_loss: 481.0395
Epoch 86/500
1144/1144 [====

KeyboardInterrupt: 

In [25]:
model.evaluate(X_test, y_test)

286/286 [==============================] - 0s 981us/step - loss: 443.3553


443.3553161621094

In [33]:
model.save("adr_model.h5")

In [31]:
test = keras.models.load_model("adr")

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_44 (Dense)             (None, 256)               131584    
_________________________________________________________________
dropout_37 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_45 (Dense)             (None, 512)               131584    
_________________________________________________________________
dropout_38 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_46 (Dense)             (None, 1024)              525312    
_________________________________________________________________
dropout_39 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_47 (Dense)             (None, 1024)             

In [32]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_44 (Dense)             (None, 256)               131584    
_________________________________________________________________
dropout_37 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_45 (Dense)             (None, 512)               131584    
_________________________________________________________________
dropout_38 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_46 (Dense)             (None, 1024)              525312    
_________________________________________________________________
dropout_39 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_47 (Dense)             (None, 1024)             